In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_filer_date = dbutils.widgets.get("p_file_date")


#### Produce Driver Standings

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

##### Find the Race  year for which  the data is reprocessed 

In [0]:
# race_result_list = spark.read.parquet(f"{presentation_folder_path}/race_results") \
#     .filter(f"file_date = '{v_filer_date}'") \
#     .select("race_year") \
#     .distinct() \
#     .collect()   

race_result_df = spark.read.format("delta").load(f"{presentation_folder_path}/race_results") \
    .filter(f"file_date = '{v_filer_date}'") 

In [0]:
# race_result_list
race_year_list = df_column_to_list(race_result_df,"race_year")

In [0]:
# race_year_list = []
# for race_year in race_result_list:
#     race_year_list.append(race_year.race_year)
# print(race_year_list)


In [0]:
from pyspark.sql.functions import col

In [0]:
race_result_df = spark.read.format("delta").load(f"{presentation_folder_path}/race_results") \
    .filter(col("race_year").isin(race_year_list))


In [0]:
# display(race_result_df)

In [0]:
from pyspark.sql.functions import sum, count, when, col

driver_standings_df = race_result_df \
    .groupBy("race_year","driver_name","driver_nationality") \
    .agg(sum("points").alias("total_points"),count(when(col("position") == 1, True)).alias("wins"))

In [0]:
display(driver_standings_df.filter("race_year == 2020"))

race_year,driver_name,driver_nationality,total_points,wins


In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, rank, asc

driver_rank_spec = Window.partitionBy("race_year").orderBy(desc("total_points"),desc("wins"))

final_df = driver_standings_df.withColumn("rank",rank().over(driver_rank_spec))

In [0]:
display(final_df.filter("race_year == 2020"))

race_year,driver_name,driver_nationality,total_points,wins,rank


In [0]:
# final_df.write.mode("overwrite").parquet(f"{presentation_folder_path}/driver_standings")

# final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_presentation.driver_standings")

# overwrite_function(final_df, 'f1_presentation', 'driver_standings', 'race_year')


In [0]:
merge_condition = "tgt.driver_name = src.driver_name and tgt.race_year = src.race_year"
merge_delta_data(final_df, "f1_presentation", "driver_standings", "race_year",presentation_folder_path, merge_condition)

In [0]:
%sql

select * from f1_presentation.driver_standings where race_year = 2021;

race_year,driver_name,driver_nationality,total_points,wins,rank
2021,Lewis Hamilton,British,44.0,1,1
2021,Max Verstappen,Dutch,43.0,1,2
2021,Lando Norris,British,27.0,0,3
2021,Charles Leclerc,Monegasque,20.0,0,4
2021,Valtteri Bottas,Finnish,16.0,0,5
2021,Daniel Ricciardo,Australian,14.0,0,6
2021,Carlos Sainz,Spanish,14.0,0,6
2021,Sergio Pérez,Mexican,10.0,0,8
2021,Pierre Gasly,French,6.0,0,9
2021,Lance Stroll,Canadian,5.0,0,10


In [0]:
%sql
select race_year,count(1) from f1_presentation.driver_standings
group by race_year
order by race_year desc;

race_year,count(1)
2021,20
2020,23
2019,20
2018,20
2017,25
2016,24
2015,22
2014,24
2013,23
2012,25


In [0]:
%sql
-- drop table  f1_presentation.driver_standings;